In [165]:
import pandas as pd
import numpy as np


In [166]:
x_raw = pd.read_csv('../BigContest/AllMergeJamDonny.csv')
x = x_raw 

In [172]:
x = x.drop(columns=['survival_time','Unnamed: 0','acc_id','playtime',
 'npc_kill',
 'solo_exp',
 'party_exp',
 'boss_monster',
 'death',
 'exp_recovery',
 'private_shop',
 'game_money_change',
 'enchant_count',
 'level',
 'random_defender_cnt',
 'num_opponent',
 'buy_item_cnt',
 'play_char_cnt',
 'temp_cnt_plg',
 'etc_cnt_plg',
 'non_combat_play_time','quest_exp',
 'revive',
 'fishing',
 'pledge_cnt',
 'temp_cnt',
 'same_pledge_cnt',
 'amount_spent_pay',
 'combat_char_cnt',
 'pledge_combat_cnt',
 'random_attacker_cnt_plg',
 'random_defender_cnt_plg','sell_item_cnt', 'combat_play_time'])
# x = x.drop(columns=['survival_time','Unnamed: 0','acc_id','class','level','random_defender_cnt','num_opponent','playtime','npc_kill','solo_exp','party_exp','boss_monster','revive','exp_recovery','private_shop','game_money_change','enchant_count','play_char_cnt','random_attacker_cnt_plg','temp_cnt_plg','etc_cnt_plg','non_combat_play_time','pledge_cnt','temp_cnt','etc_cnt','quest_exp','death','combat_char_cnt','combat_play_time','random_defender_cnt_plg','same_pledge_cnt_plg','pledge_combat_cnt','fishing'])
# x = x.drop(columns=['survival_time','Unnamed: 0','acc_id','class','level','random_defender_cnt','num_opponent','playtime','npc_kill','solo_exp','party_exp','boss_monster','death','exp_recovery','private_shop','game_money_change','pledge_combat_cnt','temp_cnt_plg','etc_cnt_plg','combat_play_time','non_combat_play_time','pledge_cnt','temp_cnt','etc_cnt','quest_exp','revive','fishing','play_char_cnt','combat_char_cnt','random_attacker_cnt_plg','same_pledge_cnt_plg'])
# x = x.drop(columns=['Unnamed: 0','acc_id','class','level','survival_time','pledge_cnt','random_attacker_cnt','random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt','num_opponent','party_exp','boss_monster','death','revive','exp_recovery','private_shop','enchant_count','play_char_cnt','combat_char_cnt','random_attacker_cnt_plg','temp_cnt_plg','non_combat_play_time'])
# ,'playtime','npc_kill','solo_exp','fishing','amount_spent_pmt','pledge_combat_cnt','game_money_change','random_defender_cnt_plg','same_pledge_cnt_plg','quest_exp'

In [188]:
x_data = [ x.iloc[i,:-1].tolist() for i in x.index.values]

In [189]:
X = x_data

In [190]:
y_data = [[i] for i in x['amount_spent'].tolist() ]

In [191]:
y= y_data

In [192]:
from sklearn.model_selection import train_test_split

x_data, X_test, y_data, y_test = train_test_split(X, y, test_size = 1/3, random_state = 42)

In [193]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_data)
data = scaler.transform(x_data)
x_data = data
# type(list(x_data))

In [194]:
scaler.fit(X_test)
data = scaler.transform(X_test)
X_test = data
# type(list(x_data))


In [195]:
x_data.shape

(26666, 3)

In [196]:
len(y_data)

26666

In [198]:
# functions.py

import tensorflow as tf
import functools, operator

def getLength(t):
    temp = (dim.value for dim in t.get_shape())         # dim is Dimension class.
    return functools.reduce(operator.mul, temp)

def showConstant(t):
    sess = tf.InteractiveSession()
    print(t.eval())
    sess.close()

def showConstantDetail(t):
    sess = tf.InteractiveSession()
    print(t.eval())
    print('shape :', tf.shape(t))
    print('size  :', tf.size(t))
    print('rank  :', tf.rank(t))
    print(t.get_shape())

    sess.close()

def showVariable(v):
    sess = tf.InteractiveSession()
    v.initializer.run()
    print(v.eval())
    sess.close()

def var2Numpy(v):
    sess = tf.InteractiveSession()
    v.initializer.run()
    n = v.eval()
    sess.close()

    return n

def op2Numpy(op):
    sess = tf.InteractiveSession()
    init = tf.initialize_all_variables()
    sess.run(init)
    ret = sess.run(op)
    sess.close()

    return ret

def showOperation(op):
    print(op2Numpy(op))


# 출처: https://pythonkim.tistory.com/62?category=574914 [파이쿵]

In [202]:
# multi_variable_linear_regression.py
# 4

import tensorflow as tf


X = tf.placeholder(tf.float32,shape=[None,x_data.shape[1]])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([x_data.shape[1],1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W) + b  #

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=3e-1)
train = optimizer.minimize(cost)



# RMSE( root-mean-square error ) : 평균 제곱 근 오차
# 평균 제곱근편차(RMSD) 또는 평균 제곱근오차(RMSE)는 추정값 또는 모델이 예측한 값과
# 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다.
# 정밀도를 표현하는데 적합하다.

targets = tf.placeholder(tf.float32,[None,1])  # 답(Y)
predictions = tf.placeholder(tf.float32,[None,1]) # 예측 값
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# start training
for step in range(10001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step, cost_val, W_val, b_val)

 # Test step
test_predict = sess.run(hypothesis,feed_dict={X:X_test})
rmse_val = sess.run(rmse,feed_dict={targets:y_test,predictions:test_predict})
print('RMSE: {}'.format(rmse_val))


total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, test_predict)))
R_squared = tf.subtract(1.0,tf.divide( unexplained_error,total_error))

print('R_sqr: {}'.format(sess.run(R_squared)))

prd = sess.run(hypothesis, feed_dict = {X:X_test})


0 0.54158634 [[-0.594726  ]
 [ 0.92756414]
 [ 1.0760803 ]] [0.11546693]
1000 0.53757215 [[0.3343682 ]
 [0.01059905]
 [0.30720237]] [0.11423689]
2000 0.5375722 [[0.3343672 ]
 [0.01059808]
 [0.30720142]] [0.1142359]
3000 0.537573 [[0.33523512]
 [0.0114635 ]
 [0.30806693]] [0.11510557]
4000 0.53757215 [[0.33436826]
 [0.01059917]
 [0.30720246]] [0.114237]
5000 0.5375722 [[0.33436635]
 [0.0105972 ]
 [0.30720052]] [0.11423502]
6000 0.53757226 [[0.33434084]
 [0.01057168]
 [0.30717504]] [0.11420951]
7000 0.5375722 [[0.33436126]
 [0.01059205]
 [0.3071954 ]] [0.11422988]
8000 0.5377123 [[0.34567308]
 [0.0219057 ]
 [0.3185095 ]] [0.12554437]
9000 0.5375722 [[0.33436814]
 [0.01059898]
 [0.30720228]] [0.11423682]
10000 0.53757226 [[0.33421212]
 [0.01044302]
 [0.30704632]] [0.11408083]
RMSE: 0.6989369988441467
R_sqr: 0.0011432170867919922


In [204]:
col = list(x.columns)

col.pop(-1)
col

['random_attacker_cnt', 'etc_cnt', 'same_pledge_cnt_plg']

In [205]:
w = pd.DataFrame(W_val)
w = w.rename(columns = {0: 'w'})
c = pd.DataFrame(col)
c = c.rename(columns = {0: 'colName'})

In [206]:
result = pd.concat([c,w],axis=1)
result

colName         w
0  random_attacker_cnt  0.334212
1              etc_cnt  0.010443
2  same_pledge_cnt_plg  0.307046

In [164]:
reTry= list(result[result['w']<0].colName)
reTry # 제외할 컬럼들

['sell_item_cnt', 'combat_play_time']

In [207]:
pd.concat([pd.DataFrame(prd),pd.DataFrame(y_test)],axis=1)

0         0
0      0.114814  0.061357
1      0.114671  0.036092
2      0.114675  0.000000
3      0.126504  0.025954
4      0.114671  0.000000
...         ...       ...
13329  0.114671  0.000000
13330  0.129481  0.096071
13331  0.119744  0.000000
13332  0.114671  0.000000
13333  0.162850  0.168222

[13334 rows x 2 columns]

In [ ]:
# 0.001 (1e-3)-> 
# 0.003 (3e-3)-> 
# 0.01 (1e-2)-> 
# 0.03 (3e-2)-> 
# 0.1 (1e-1)-> 
# 0.3 (3e-1)-> 
# 1